In [ ]:
from typing import List, Tuple, Union

import numpy as np
from PIL import Image, ImageDraw

import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib.pyplot import colormaps
from matplotlib.animation import FuncAnimation

from ee23b035_draw import draw_keyboard
from ee23b035_keyboard import REQUIRED_NORMAL_KEYS, REQUIRED_SPECIAL_KEYS
from ee23b035_simulated_annealing import simulated_annealing, cost_function

from ee23b035_config import KB_LAYOUT, HOME_ROW_POS, KB_Type
from ee23b035_config import KB_W, KB_H, G_WIDTH, K_PRESS_HMAP_W, CMAP

from ee23b035_utils import Position
from ee23b035_utils import find_key_name, get_key_position
from ee23b035_utils import generate_kb_rows, euclidean_dist

In [ ]:
def type_key(char: str, KB_LAYOUT: KB_Type) -> List[str]:
    """
    Return list of keys to be pressed in order to type a character

    Args:
        char (str): Character to be type

    Returns:
        List[str]: List of key names to be pressed
    """

    # Map special characters to the key names
    special_chars = {" ": "Space", "\n": "Enter", "\t": "Tab", "\b": "Backspace"}
    # If the given char is a special char, update the name
    char = special_chars.get(char, char)

    pos, USE_SHIFT = get_key_position(char, KB_LAYOUT)
    KEY_LEFT = True  # Indicate which side of the keyboard the key is on

    # We have to press the key containing char
    yield find_key_name(char, KB_LAYOUT)
    
    num_home_row_keys = len(HOME_ROW_POS)
    col_middle = HOME_ROW_POS[num_home_row_keys // 2][1] + HOME_ROW_POS[num_home_row_keys // 2 - 1][1]
    col_middle = col_middle / 2

    # Account for distance to travel to shift key
    if USE_SHIFT:
        if pos[1] < col_middle:
            # If key is on left, type shift with right hand
            yield "Shift_R" # Also press shift
        else:
            # If key is on right, type shift with left hand
            yield "Shift_L" # Also press shift

In [ ]:
IGNORE_SPACES = True
INIT_TEMP = 1000

In [ ]:
text = """
Give which dominion i meat grass spirit gathering, subdue years night. From they're together grass thing said isn't divided.
Unto, hath likeness, he they're all for you yielding heaven tree together winged they're.
Earth you'll forth multiply under days first the moved moveth he be earth fruit upon him morning.
Gathered Unto, in won't above itself female fifth fruit from. Winged make. From beginning.
Earth first without waters beast together, was the green from let unto of him years and image said so.
Moved you deep. Of male multiply every you're forth said after darkness seas gathering is fruit you shall
two together feline can't their void cattle after that wherein land blessed i wherein dry without saying over, greater subdue blessed hath.
Abundantly given wherein greater gopher for behold. Moved. He there bearing stars multiply created doesn't one.
Moved from fowl male own after said seed. Sixth, bring beast life you're green upon. Fish. Bearing.
Saw hath their. Air. Green be replenish whales sixth First. Had. In to cattle saying he God moved face from thing and.
The he upon great shall moveth. Called moveth cattle moveth called don't without kind firmament may image second brought.
Moved greater creeping.
123456789012345678901234567890123456789012345678901234567890
123456789012345678901234567890123456789012345678901234567890
123456789012345678901234567890123456789012345678901234567890
123456789012345678901234567890123456789012345678901234567890
123456789012345678901234567890123456789012345678901234567890
123456789012345678901234567890123456789012345678901234567890
123456789012345678901234567890123456789012345678901234567890
"""

if IGNORE_SPACES:
    text = text.replace(" ", "")

In [ ]:
# First, calculate the travel_dist for the unoptimized layout
unoptimized_travel_dist = cost_function(text, KB_LAYOUT)
print(f"Unoptimized Travel Distance {unoptimized_travel_dist:.3f}")

best_dist_dict = {}
curr_dist_dict = {}

counter = 0

# Optimize the keyboard layout
for best_dist, curr_dist, temp in simulated_annealing(text, INIT_TEMP, KB_LAYOUT):
    print(f"Temperature: {temp:04}", end="\r")
    best_dist_dict[counter] = best_dist
    curr_dist_dict[counter] = curr_dist
    counter += 1

travel_dist = cost_function(text, KB_LAYOUT)
print(f"Optimized Travel Distance {travel_dist:.3f}")

In [ ]:
mpl.rcParams['animation.embed_limit'] = 2**10

fig, ax = plt.subplots()
xdata, best_cost_data, curr_cost_data = [0], [unoptimized_travel_dist], [unoptimized_travel_dist]
ln_0, = ax.plot([], [], 'r')
ln_1, = ax.plot([], [], 'g')

ax.legend(["Best Cost", "Instantaneous Cost"])
plt.title("Simulated Annealing for Keyboard Optimization")

def init():
    ax.set_xlim(0, counter)
    ax.set_ylim(0, max(curr_dist_dict.values()) * 1.1)
    return ln_0, ln_1

def update(frame):
    xdata.append(frame)
    best_cost_data.append(best_dist_dict[frame])
    curr_cost_data.append(curr_dist_dict[frame])
    ln_0.set_data(xdata, best_cost_data)
    ln_1.set_data(xdata, curr_cost_data)
    return ln_0, ln_1

ani = FuncAnimation(fig, update, frames=range(counter),
                    init_func=init, blit=True, interval=10, repeat=False)

from IPython.display import HTML
HTML(ani.to_jshtml())